# AI算法性能对比分析 (Person E)

本笔记本对比分析不同AI算法的性能表现

In [ ]:
# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 设置样式
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

print("✓ 环境准备完成")

## 1. 加载分析结果

In [ ]:
# 加载各项分析结果
win_rates = pd.read_csv('../data/results/win_rates.csv')
time_stats = pd.read_csv('../data/results/response_times.csv')
elo_ratings = pd.read_csv('../data/results/elo_ratings.csv')
matchup_matrix = pd.read_csv('../data/results/matchup_matrix.csv', index_col=0)

print(f"✓ 加载完成")
print(f"  评估算法数: {len(win_rates)}")

## 2. 算法列表与特点

In [ ]:
# 合并所有指标
comparison = win_rates.merge(time_stats, on='algorithm')
comparison = comparison.merge(elo_ratings, on='algorithm')

# 选择关键列显示
display_cols = ['algorithm', 'win_rate', 'mean_time', 'elo_rating', 'wins', 'losses']
comparison[display_cols]

## 3. 多维度性能对比

### 3.1 胜率排名

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colors = sns.color_palette('RdYlGn', len(win_rates))
ax.barh(win_rates['algorithm'], win_rates['win_rate'], color=colors)

for i, (idx, row) in enumerate(win_rates.iterrows()):
    ax.text(row['win_rate'] + 0.02, i, f"{row['win_rate']:.1%}", 
           va='center', fontsize=12fontweight='bold')

ax.set_xlabel('胜率', fontsize=14)
ax.set_title('算法胜率排名', fontsize=16, fontweight='bold')
ax.set_xlim(0, 1)
plt.tight_layout()
plt.show()

### 3.2 ELO评分对比

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colors = sns.color_palette('viridis', len(elo_ratings))
ax.bar(range(len(elo_ratings)), elo_ratings['elo_rating'], color=colors)
ax.set_xticks(range(len(elo_ratings)))
ax.set_xticklabels(elo_ratings['algorithm'])

# 添加数值标签
for i, row in elo_ratings.iterrows():
    ax.text(i, row['elo_rating'] + 10, f"{row['elo_rating']:.0f}", 
           ha='center', fontsize=12)

ax.set_ylabel('ELO评分', fontsize=14)
ax.set_title('算法ELO评分对比', fontsize=16, fontweight='bold')
ax.axhline(1500, color='red', linestyle='--', alpha=0.5, label='初始ELO')
ax.legend()
plt.tight_layout()
plt.show()

### 3.3 响应时间对比

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(time_stats))
width = 0.35

ax.bar(x - width/2, time_stats['mean_time'], width, label='平均时间', alpha=0.8)
ax.bar(x + width/2, time_stats['median_time'], width, label='中位数时间', alpha=0.8)

ax.set_ylabel('时间 (秒)', fontsize=14)
ax.set_title('响应时间对比', fontsize=16, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(time_stats['algorithm'])
ax.legend()
plt.tight_layout()
plt.show()

## 4. 对战矩阵分析

In [ ]:
# 显示对战矩阵
print("对战矩阵 (行 vs 列的胜率):")
matchup_matrix

In [ ]:
# 可视化对战矩阵
plt.figure(figsize=(10, 8))
sns.heatmap(matchup_matrix, annot=True, fmt='.2f', cmap='RdYlGn', 
           center=0.5, vmin=0, vmax=1, square=True, linewidths=1,
           cbar_kws={'label': '胜率'})
plt.title('算法对战矩阵', fontsize=16, fontweight='bold')
plt.ylabel('算法 (行)', fontsize=14)
plt.xlabel('对手 (列)', fontsize=14)
plt.tight_layout()
plt.show()

## 5. 综合评分与排名

In [ ]:
# 标准化各项指标（0-1范围）
comparison['win_rate_norm'] = comparison['win_rate']
comparison['speed_norm'] = 1 - (comparison['mean_time'] - comparison['mean_time'].min()) / \
                              (comparison['mean_time'].max() - comparison['mean_time'].min() + 0.001)
comparison['elo_norm'] = (comparison['elo_rating'] - comparison['elo_rating'].min()) / \
                         (comparison['elo_rating'].max() - comparison['elo_rating'].min() + 0.001)

# 计算综合得分（可调整权重）
weights = {'win_rate': 0.5, 'speed': 0.2, 'elo': 0.3}
comparison['综合得分'] = (comparison['win_rate_norm'] * weights['win_rate'] + 
                      comparison['speed_norm'] * weights['speed'] + 
                      comparison['elo_norm'] * weights['elo'])

# 排序
comparison_sorted = comparison.sort_values('综合得分', ascending=False)
comparison_sorted[['algorithm', 'win_rate', 'mean_time', 'elo_rating', '综合得分']]

In [ ]:
# 综合得分可视化
fig, ax = plt.subplots(figsize=(10, 6))

colors = sns.color_palette('coolwarm', len(comparison_sorted))
ax.barh(comparison_sorted['algorithm'], comparison_sorted['综合得分'], color=colors)

for i, (idx, row) in enumerate(comparison_sorted.iterrows()):
    ax.text(row['综合得分'] + 0.01, i, f"{row['综合得分']:.3f}", 
           va='center', fontsize=12)

ax.set_xlabel('综合得分', fontsize=14)
ax.set_title(f"算法综合评分排名\n(权重: 胜率{weights['win_rate']}, 速度{weights['speed']}, ELO{weights['elo']})", 
            fontsize=16, fontweight='bold')
ax.set_xlim(0, 1)
plt.tight_layout()
plt.show()

## 6. 效率-性能权衡分析

In [ ]:
# 散点图：响应时间 vs 胜率
fig, ax = plt.subplots(figsize=(10, 6))

scatter = ax.scatter(comparison['mean_time'], comparison['win_rate'], 
                    s=200, c=comparison['elo_rating'], 
                    cmap='viridis', alpha=0.7, edgecolors='black', linewidth=2)

# 添加算法标签
for _, row in comparison.iterrows():
    ax.annotate(row['algorithm'], 
               (row['mean_time'], row['win_rate']),
               xytext=(5, 5), textcoords='offset points', fontsize=11)

ax.set_xlabel('平均响应时间 (秒)', fontsize=14)
ax.set_ylabel('胜率', fontsize=14)
ax.set_title('算法效率-性能权衡', fontsize=16, fontweight='bold')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
ax.grid(True, alpha=0.3)

# 添加理想区域标注
ax.axhline(0.5, color='red', linestyle='--', alpha=0.3, label='50%胜率线')
ax.legend()

plt.colorbar(scatter, label='ELO评分')
plt.tight_layout()
plt.show()

## 7. 推荐最佳算法

In [ ]:
print("="*60)
print(" 算法推荐")
print("="*60)

print(f"\n🏆 最高胜率: {win_rates.iloc[0]['algorithm']}")
print(f"   - 胜率: {win_rates.iloc[0]['win_rate']:.1%}")
print(f"   - 适用场景: 追求最高胜率，不考虑响应时间")

print(f"\n⚡ 最快响应: {time_stats.iloc[0]['algorithm']}")
print(f"   - 响应时间: {time_stats.iloc[0]['mean_time']:.4f}s")
print(f"   - 适用场景: 实时对战，需要快速响应")

print(f"\n⭐ 综合最佳: {comparison_sorted.iloc[0]['algorithm']}")
print(f"   - 综合得分: {comparison_sorted.iloc[0]['综合得分']:.3f}")
print(f"   - 胜率: {comparison_sorted.iloc[0]['win_rate']:.1%}")
print(f"   - 响应时间: {comparison_sorted.iloc[0]['mean_time']:.4f}s")
print(f"   - ELO: {comparison_sorted.iloc[0]['elo_rating']:.0f}")
print(f"   - 适用场景: 平衡性能与效率的通用场景")

print("\n" + "="*60)

## 8. 各算法适用场景分析

In [ ]:
# 创建场景推荐表
scenarios = []

for _, row in comparison.iterrows():
    algo = row['algorithm']
    
    # 根据特性判断适用场景
    if row['win_rate'] > 0.7:
        strength = "高胜率"
        scenario = "竞技对战、AI训练对手"
    elif row['mean_time'] < 0.01:
        strength = "极快响应"
        scenario = "实时游戏、移动端应用"
    elif 0.5 < row['win_rate'] < 0.7 and row['mean_time'] < 1:
        strength = "平衡性能"
        scenario = "通用对战、新手挑战"
    else:
        strength = "基准算法"
        scenario = "性能对比基准"
    
    scenarios.append({
        '算法': algo,
        '特点': strength,
        '推荐场景': scenario,
        '胜率': f"{row['win_rate']:.1%}",
        '响应时间': f"{row['mean_time']:.4f}s"
    })

scenario_df = pd.DataFrame(scenarios)
scenario_df

## 9. 总结与建议

### 主要发现

1. **性能层次明显**: 不同算法在胜率上存在显著差异
2. **效率差异巨大**: 响应时间从毫秒级到秒级不等
3. **存在权衡**: 高胜率算法通常响应较慢

### 使用建议

- **追求胜率**: 选择胜率最高的算法
- **追求速度**: 选择响应最快的算法
- **综合考虑**: 根据综合评分选择
- **特定场景**: 参考场景推荐表

### 未来改进方向

1. 引入深度学习算法（DQN、PPO等）
2. 优化现有算法的搜索效率
3. 实现算法难度自适应调整
4. 开发混合策略算法